# YOLO-based Segmentation of Recoater Streaking & Spatter

This notebook uses a pre-trained YOLOv8 segmentation model to detect recoater streaks and spatter
in each layer of a Laser Powder Bed Fusion build. We then compute per-layer anomaly areas for
downstream analysis.

In [1]:
import logging
import os

import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import yaml
from src.utils.yolo_segment import (
    load_hdf5_slice,
    load_hdf5_stack,
)
from ultralytics import YOLO

### Set Up

In [2]:
BASE = os.path.abspath("../data/tcr_phase1_build2")
DATA_PATH     = os.path.abspath("../data/2021-04-16 TCR Phase 1 Build 2.hdf5") 
CAMERA_PATH   = "slices/camera_data/visible/0"
IMG_TRAIN_DIR = os.path.abspath("../data/tcr_phase1_build2/images/train")
LBL_TRAIN_DIR = os.path.abspath("../data/tcr_phase1_build2/labels/train") 
DATA_YAML     = os.path.abspath("data.yaml") 

# Classes
CLASS_MAP = {1: "spatter", 2: "streak"}
PIXEL_SIZE_MM2 = 0.01  # adjust to your calibration

# YOLO parameters
EPOCHS       = 5
BATCH_SIZE   = 2
IMG_SIZE     = 256
CONF_THRESH  = 0.25

In [3]:
cfg = {
    "path": BASE,           # root for both images/ and labels/ (or masks/)
    "train": "images/train", 
    "val":   "images/val",
    "nc":    len(CLASS_MAP),
    "names": list(CLASS_MAP.values()),
}

with open(DATA_YAML, "w") as f:
    yaml.dump(cfg, f, sort_keys=False)
print("Wrote", DATA_YAML)

Wrote /piml-in-metal-am/notebooks/data.yaml


### Training with YOLO v11

In [4]:
if not torch.cuda.is_available():
	raise RuntimeError("CUDA is not available. Please check your installation.")

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

logger.info("Starting YOLO model training...")
try:
    model = YOLO("yolo11n-seg.pt")
    
    model.train(
    data="data.yaml",
    epochs=5,
    imgsz=256,
    batch=2,
    cache=False,        # **critical**
    workers=2,
    device=0,           # GPU 0
    amp=True,
    close_mosaic=0,
    plots=False,
    )
    logger.info("Training completed successfully.")
except Exception as e:
    logger.error("An error occurred during training:", exc_info=True)

INFO:__main__:Starting YOLO model training...


Ultralytics 8.3.140 🚀 Python-3.13.3 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 2070 with Max-Q Design, 7787MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=2, bgr=0.0, box=7.5, cache=disk, cfg=None, classes=None, close_mosaic=1, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/piml-in-metal-am/notebooks/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=True, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo_v116, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=

train: Scanning /piml-in-metal-am/data/tcr_phase1_build2/labels/train.cache... 3027 images, 5 backgrounds, 0 corrupt: 100%|██████████| 3027/3027 [00:00<?, ?it/s]
train: Caching images (28.7GB Disk): 100%|██████████| 3027/3027 [00:00<00:00, 44398.06it/s]

WARNING ⚠️ 'rect=True' is incompatible with DataLoader shuffle, setting shuffle=False


val: Fast image access ✅ (ping: 0.1±0.2 ms, read: 198.2±30.3 MB/s, size: 480.3 KB)


val: Scanning /piml-in-metal-am/data/tcr_phase1_build2/labels/val.cache... 534 images, 1 backgrounds, 0 corrupt: 100%|██████████| 534/534 [00:00<?, ?it/s]
val: Caching images (5.1GB Disk): 100%|██████████| 534/534 [00:00<00:00, 51383.57it/s]


Plotting labels to ../runs/segment/yolo_v116/labels.jpg... 


### Sanity Check

In [ ]:
imgs = load_hdf5_stack(H5_PATH, "slices/camera_data/visible/0")[:4]  # first 4 layers

# run YOLOv11-Seg
results = predict(model, imgs, imgsz=640, conf=0.25)

# overlay first result
from matplotlib import pyplot as plt
plt.figure(figsize=(6,6))
plt.imshow(overlay_results(imgs[0], results[0]))
plt.axis("off")
plt.show()

In [ ]:
finetuned = YOLO("runs/segment/yolo_spatter_streak/weights/best.pt")
# test on layer 0
with h5py.File(DATA_PATH, "r") as h5:
    img0 = load_hdf5_slice(DATA_PATH, 0, CAMERA_PATH)
res0 = finetuned(img0, imgsz=IMG_SIZE, conf=CONF_THRESH)
vis0 = visualize_detections(img0, res0)
plt.imshow(vis0); plt.axis("off"); plt.title("Layer 0 - Fine-tuned")

In [ ]:
# 1) load full stack into memory (or chunk manually if too big)
stack = load_hdf5_stack(DATA_PATH, CAMERA_PATH)  # shape (N,H,W,3)
N = stack.shape[0]

# 2) preallocate
recoater_areas = np.zeros(N, dtype=float)
spatter_areas  = np.zeros(N, dtype=float)

# 3) run in batches
for i in range(0, N, BATCH_SIZE):
    batch = stack[i : i + BATCH_SIZE]
    recoater_areas[i : i + BATCH_SIZE] = batch_predict_and_compute_areas(
        finetuned, batch, [2], PIXEL_SIZE_MM2, imgsz=IMG_SIZE, conf=CONF_THRESH
    )
    spatter_areas[i : i + BATCH_SIZE] = batch_predict_and_compute_areas(
        finetuned, batch, [1], PIXEL_SIZE_MM2, imgsz=IMG_SIZE, conf=CONF_THRESH
    )
    print(f"Processed layers {i}–{i+BATCH_SIZE}")

# 4) assemble DataFrame
df = pd.DataFrame({
    "layer": np.arange(N),
    "recoater_mm2": recoater_areas,
    "spatter_mm2": spatter_areas,
})
df.head()

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(df.layer, df.recoater_mm2, label="Recoater")
plt.plot(df.layer, df.spatter_mm2, label="Spatter")
plt.xlabel("Layer"); plt.ylabel("Anomaly Area (mm²)")
plt.legend(); plt.title("Per-Layer Anomaly Areas")
plt.show()